# Using the `xarray` interface

Performing optimizations in multiple dimensions is a part of life. Luckily this can be done as easily as the previous example.

In [ ]:
from easyCore import np
from easyCore.Datasets.xarray import xr
from easyCore.Objects.ObjectClasses import Parameter, BaseObj
from easyCore.Fitting.Fitting import Fitter
import holoviews as hv
import matplotlib.pyplot as plt
hv.extension('bokeh')

## Creating a dataset

We will creat some dummy data, in this case a 2D plot of Sine's and Cose's

In [ ]:
d = xr.Dataset()

nx = 5E2
x_min = 0
x_max = np.pi

sin_offest = - 0.35
cos_offset = 0.14

x = np.linspace(x_min, x_max, num=int(nx))
y = x + 1

d.easyCore.add_coordinate('x', x)
d.easyCore.add_coordinate('y', y)
d.easyCore.add_variable('z', ['x', 'y'], np.sin(2*np.pi*(x + sin_offest)).reshape((-1, 1))*np.cos(2*np.pi*(x + cos_offset)).reshape((1, -1)) + 2*(0.5 - np.random.random(size=(int(nx), int(nx)))))

s_off_start_point = 0
c_off_start_point = 0

## Make the object and perform a fit

In [ ]:
b = BaseObj('SinCos2D',
            s_off=Parameter('s_off', s_off_start_point),
            c_off=Parameter('c_off', c_off_start_point))


def fit_fun(x, *args, **kwargs):
    # In the real case we would gust call the evaluation fn without reference to the BaseObj
    return np.sin(2*np.pi*(x[:, 0] + b.s_off.raw_value)) * np.cos(2*np.pi*(x[:, 1] + b.c_off.raw_value))


f = Fitter()
f.initialize(b, fit_fun)

f_res = d['z'].easyCore.fit(f)

In [ ]:
d['computed'] = f_res.y_calc
temp = d['z'] - d['computed']

print(f_res.p0)
print(f_res.p)

fig, axes = plt.subplots(ncols=2, nrows=2)
d['z'].plot(ax=axes[0, 0])
d['computed'].plot(ax=axes[0, 1])
temp.plot(ax=axes[1, 0])
plt.tight_layout()
plt.show()